<h1 align="center">Лабораторна робота №3<br>Використання декораторів, користувацьких виключень та магічних методів</h1>

***<center>Сухонос Вероніка, КН-423к</center>***

**Мета роботи:** отримати базові навички роботи з абстрактними класами, декораторами, користувацькими виключеннями та магічними методами у мові програмування Python.

# Завдання

Створити абстрактний клас `Vehicle` та класи `Car`, `Motorcycle`, `Truck` і `Bus`, успадковані від `Vehicle`.

Клас `Vehicle` приймає такі параметри:
* `brand_name` -> str (наприклад, Honda).
* `year_of_issue` -> int (наприклад, 2024).
* `base_price` -> int (наприклад, 100000).
* `mileage` -> int (наприклад, 10000).

Необхідно створити такі методи:
* `vehicle_type` - повертає str - тип транспортного засобу за шаблоном: **brand_name + назва класу** (наприклад: Toyota Car, Suzuki Motorcycle).
* `is_motorcycle` - повертає bool, що залежить від кількості коліс (2 колеса -> мотоцикл -> True).
* `purchase_price` - повертає float - ціну транспортного засобу за формулою **base_price - 0,1 * mileage**; якщо кінцева ціна менша за 5000, метод має повернути 5000.

Поставити наступні декоратори, де необхідно і якщо це необхідно: `abstractmethod`, `classmethod`, `staticmethod`, `property` тощо.

## Додаткові завдання:

1. Валідація:
    * Перевірка на допустимі межі:
        - `year_of_issue` у діапазоні припустимих років (наприклад, від 1900 року до поточного року).
        - `base_price` та `mileage` не менше нуля (або відповідають іншим вашим вимогам щодо мінімальних значень).
    * Перевірка типу даних:
        - `year_of_issue`, `base_price` та `mileage` є цілими числами (integers) або числами з плаваючою точкою (float), залежно від ваших потреб.
    * Перевірка на коректність даних:
        - для `year_of_issue` рік не перевищує поточний рік і не менше за допустиму межу, якщо це відповідає вашим вимогам.
        - для `base_price` ціна не є негативною, якщо це відповідає вашим вимогам.
        - для `mileage` пробіг не є негативним, якщо це відповідає вашим вимогам.
    * Перевірка на цілочисельність:
        - якщо `base_price` або `mileage` має бути цілим числом, вони не мають містити дробової частини або інших символів, якщо це відповідає вашим вимогам.
    * Додаткові вимоги:
        - будь-які додаткові вимоги валідації, що відповідають вашим конкретним потребам або бізнес-правилам (наприклад, перевірка на максимально допустимий пробіг або максимальну ціну).

2. Створити методи для доступу до приватних та захищених змінних:
    - додати методи (геттери та сеттери) для доступу до приватних та захищених змінних.
    - використати декоратори `@property`, `@<property_name>.setter`, а також інші необхідні декоратори, які дозволяють контролювати доступ до цих змінних.

3. Створити альтернативний конструктор (наприклад, на основі рядка, який містить розділені комами значення параметрів).

4. Переробити оператор додавання в класі `Car` - при додаванні двох екземплярів класу повинно розрахувати сумарний пробіг та базову вартість.

In [3]:
from abc import ABC, abstractmethod
from datetime import datetime

In [4]:
class MissingParameterError(ValueError):
  def __init__(self, param):
    super().__init__(f'{param} is required')

class ValidationError(ValueError):
  pass

In [5]:
class Vehicle(ABC):
  MIN_PRICE = 5000
  MIN_YEAR_OF_ISSUE = 1900
  MAX_BASE_PRICE = 1_000_000_000
  MAX_MILEAGE = 10_000_000
  MAX_WHEELS = 10

  def __init__(
    self,
    brand_name: str = None,
    year_of_issue: int = None,
    base_price: int = None,
    mileage: int = None
  ):
    self.brand_name = brand_name
    self.year_of_issue = year_of_issue
    self.base_price = base_price
    self.mileage = mileage
    self.wheels = self.default_wheels()

  def vehicle_type(self):
    return f'{self.brand_name} {self.__class__.__name__}'

  @abstractmethod
  def default_wheels(self):
    pass

  @property
  def wheels(self) -> int:
    return self._wheels

  @wheels.setter
  def wheels(self, val):
    if val is None:
      raise MissingParameterError('wheels')
    self._wheels = Vehicle.is_valid_integer(val, 'wheels', self.MAX_WHEELS, 1)

  def is_motorcycle(self):
    return self.wheels == 2

  @staticmethod
  def is_valid_integer(val, param, to, fr):
    try:
      val_int = int(val)
    except:
      raise ValidationError(f'{param} must be a valid integer')
    if not (fr <= val_int <= to):
      raise ValidationError(
        f'{param} must be an integer from {fr} to {to}'
      )
    return val_int

  @property
  def purchase_price(self):
    return max(self.base_price - 0.1 * self.mileage, self.MIN_PRICE)

  @property
  def brand_name(self):
    return self._brand_name

  @brand_name.setter
  def brand_name(self, val):
    if val is None:
      raise MissingParameterError('brand_name')
    if type(val) is not str:
      raise ValidationError('brand_name must be a string')
    self._brand_name = Vehicle.sanitize_brand_name(val)

  @staticmethod
  def sanitize_brand_name(name):
    return ' '.join([w for w in name.split(' ') if w.strip() != ''])

  @property
  def year_of_issue(self):
    return self._year_of_issue

  @year_of_issue.setter
  def year_of_issue(self, val):
    if val is None:
      raise MissingParameterError('year_of_issue')
    now = datetime.now().year
    self._year_of_issue = Vehicle.is_valid_integer(val, 'year_of_issue', now, self.MIN_YEAR_OF_ISSUE)

  @property
  def base_price(self):
    return self._base_price

  @base_price.setter
  def base_price(self, val):
    if val is None:
      raise MissingParameterError('base_price')
    self._base_price = Vehicle.is_valid_integer(val, 'base_price', self.MAX_BASE_PRICE, 0)

  @property
  def mileage(self):
    return self._mileage

  @mileage.setter
  def mileage(self, val):
    if val is None:
      raise MissingParameterError('mileage')
    self._mileage = Vehicle.is_valid_integer(val, 'mileage', self.MAX_MILEAGE, 0)

  @classmethod
  def from_str(cls, constr_string):
    if constr_string is None:
      raise MissingParameterError('constr_string')
    if type(constr_string) is not str:
      raise ValidationError('constr_string must be a string')
    params = [p.strip() for p in constr_string.split(',')]
    if len(params) != 4:
      raise ValidationError(
        'constr_string must have the following format: brand_name,year_of_issue,base_price,mileage'
      )
    return cls(*params)

In [6]:
class Car(Vehicle):

  def default_wheels(self):
    return 4

  def __add__(self, other):
    return (self.base_price + other.base_price, self.mileage + other.mileage)

In [7]:
class Motorcycle(Vehicle):

  def default_wheels(self):
    return 2

In [8]:
class Truck(Vehicle):

  def default_wheels(self):
    return 4

In [9]:
class Bus(Vehicle):

  def default_wheels(self):
    return 4

In [10]:
vehicles = (
  Car(brand_name="Toyota", year_of_issue=2020, base_price=1_000_000, mileage=150_000),
  Motorcycle(brand_name="Suzuki", year_of_issue=2015, base_price=800_000, mileage=35_000),
  Truck(brand_name="Scania", year_of_issue=2018, base_price=15_000_000, mileage=850_000),
  Bus(brand_name="MAN", year_of_issue=2000, base_price=10_000_000, mileage=950_000)
)

for vehicle in vehicles:
  print(
    f"Vehicle type={vehicle.vehicle_type()}\n"
    f"Is motorcycle={vehicle.is_motorcycle()}\n"
    f"Purchase price={vehicle.purchase_price}\n"
  )

Vehicle type=Toyota Car
Is motorcycle=False
Purchase price=985000.0

Vehicle type=Suzuki Motorcycle
Is motorcycle=True
Purchase price=796500.0

Vehicle type=Scania Truck
Is motorcycle=False
Purchase price=14915000.0

Vehicle type=MAN Bus
Is motorcycle=False
Purchase price=9905000.0



In [11]:
try:
  wrong_car = Car()
except ValueError as e:
  print(e)

brand_name is required


In [12]:
try:
  wrong_car = Car(brand_name="Toyota", year_of_issue=2061, base_price=1_000_000, mileage=150_000)
except ValueError as e:
  print(e)

year_of_issue must be an integer from 1900 to 2025


In [13]:
try:
  wrong_car = Car(brand_name="Toyota", year_of_issue=2024, base_price=-1_000_000, mileage=150_000)
except ValueError as e:
  print(e)

base_price must be an integer from 0 to 1000000000


In [14]:
try:
  wrong_car = Car(brand_name="Toyota", year_of_issue=2025, base_price=1_000_000, mileage='no')
except ValueError as e:
  print(e)

mileage must be a valid integer


In [15]:
car1 = Car.from_str("Toyota,2020,1_000_000,150_000")
car2 = Car.from_str("Kia,2015,800_000,35_000")

print(f'{car1 + car2}')

(1800000, 185000)


In [16]:
try:
  wrong_car = Car.from_str("2015,800_000,35_000")
except ValueError as e:
  print(e)

constr_string must have the following format: brand_name,year_of_issue,base_price,mileage
